In [27]:
!pip install datasets
!pip install faiss-cpu
!pip install ollama
!pip install -qU langchain-qdrant


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [28]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED   
nomic-embed-text:latest	0a109f422b47	274 MB	2 days ago	
llama3.1:latest        	42182419e950	4.7 GB	3 days ago	


In [29]:
!pip install langchain-community


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [50]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_qdrant import QdrantVectorStore
from qdrant_client.http.models import Distance

In [31]:
dataset_name = "databricks/databricks-dolly-15k"
page_content_column = "context"  # or any other column you're interested in

# Create a loader instance
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)

# Load the data
data = loader.load()

# Display the first 15 entries
data[:2]

[Document(metadata={'instruction': 'When did Virgin Australia start operating?', 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}, page_content='"Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia\'s domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney."'),
 Document(metadata={'instruction': 'Which is a species of fish? Tope or Rope', 'response': 'Tope', 'category': 'classification'}, page_content='""')]

In [32]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(data)

In [33]:
embedding = OllamaEmbeddings(model="nomic-embed-text")

In [34]:
embedding.embed_query("who's Dawid")

[-0.5979874134063721,
 0.1774091273546219,
 -3.693427085876465,
 -0.8051135540008545,
 -0.4065955877304077,
 0.6764993071556091,
 0.4356774091720581,
 0.6609154343605042,
 -0.6599831581115723,
 -1.3692622184753418,
 -0.8964322805404663,
 0.311566025018692,
 1.480530858039856,
 0.3602171242237091,
 0.672015368938446,
 0.3034931719303131,
 -1.8650704622268677,
 -0.13666824996471405,
 0.6001240015029907,
 0.45960119366645813,
 -1.0374701023101807,
 -0.7831551432609558,
 -0.3398086726665497,
 0.05481716990470886,
 2.319247245788574,
 0.31265100836753845,
 0.20493808388710022,
 0.24365833401679993,
 -0.5427579283714294,
 0.9496626853942871,
 0.3691500723361969,
 0.9597890377044678,
 -1.3616770505905151,
 0.9006558060646057,
 0.6429985165596008,
 -0.05966632068157196,
 0.5871962308883667,
 -0.5402980446815491,
 0.6367090940475464,
 -0.3764284551143646,
 -0.18689873814582825,
 0.21368439495563507,
 1.1099529266357422,
 -1.3012080192565918,
 0.3534037470817566,
 -0.5531500577926636,
 -0.685508

In [35]:
vecstore = QdrantVectorStore.from_documents(
    docs,
    embedding=embedding,
    location=":memory:",
    collection_name="my_documents",
    distance=Distance.COSINE,
)

In [36]:
retriever = vecstore.as_retriever()

In [37]:
vecstore.similarity_search(query="flying australia", k=4)

[Document(metadata={'instruction': 'When did Virgin Australia start operating?', 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa', '_id': 'c575496c39774c97b02c6190492ef54c', '_collection_name': 'my_documents'}, page_content='"Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia\'s domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney."'),
 Document(metadata={'instruction': 'Note down some details about Munda Airport from the given text', 'response': "1. Munda Airport in Solomon Islands' Western Pr

In [38]:
model = "llama3.1"
llm = ChatOllama(model=model)

In [39]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [40]:
# prompt = hub.pull("rlm/rag-prompt")

In [41]:
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [42]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [43]:
question = "Who is Thomas Jefferson? Write in Polish"
rag_chain.invoke(question)

'Thomas Jefferson był amerykańskim politykiem, dyplomatą i prawnikiem, który służył jako trzeci prezydent Stanów Zjednoczonych od 1801 do 1809 roku. Był również autorem Deklaracji Niepodległości. Początkowo pełnił funkcje sekretarza stanu i wiceprezydenta Stanów Zjednoczonych.\n\nThanks for asking!'

In [44]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [45]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [48]:
chat_history = []

question = "Tell me about airlines from Australia"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])
ai_msg_1['answer']

'There are several notable Australian airlines. Virgin Australia is the largest airline by fleet size to use the Virgin brand and has directly served 32 cities in Australia since its inception in 2000. It also experienced significant growth after the collapse of Ansett Australia in 2001.'

In [49]:
second_question = "Could you translate it to polish language?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Here's a translation:

"Są kilka wybitnych linii lotniczych z Australii. Virgin Australia jest największą linią lotniczą w ramach marki Virgin i obsługuje bezpośrednio 32 miast w Australii od momentu swojego powstania w 2000 roku. Linia doświadczyła istotnego wzrostu po upadku Ansett Australia w 2001 roku."
